<a href="https://colab.research.google.com/github/Eddiebee/AI-Craft/blob/main/Behind_the_Pipeline_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
raw_inputs = [
    "How I long to see your face my beauty",
    "Oh, how greatly I miss you not"
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True,
                   return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 2129, 1045, 2146, 2000, 2156, 2115, 2227, 2026, 5053,  102],
        [ 101, 2821, 1010, 2129, 6551, 1045, 3335, 2017, 2025,  102,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}


Using the AutoModel class to instantiate our model from the same checkpoint.

In [6]:
from transformers import AutoModel

model = AutoModel.from_pretrained(checkpoint)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

We can see the high-dimensionality of our model ouputs by feeding our preprocessed inputs to our model.

In [7]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 11, 768])


The 🤗 transformers comes with different architectures that are suitable for different tasks.

* *Model (rertrieve the hidden states)
* *ForCausalLM
* *ForMaskedLM
* *ForMultipleChoice
* *ForQuestionAnswering
* *ForSequenceClassification
* *ForTokenClassification

In [8]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [11]:
outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([2, 2])


The values we get from our model's output doesn't make sense by themselves.

In [12]:
outputs.logits

tensor([[-4.0520,  4.3644],
        [ 1.6896, -1.4340]], grad_fn=<AddmmBackward0>)

...logits are not actually probabilities, hence the have to be passed to the Softmax activation layer to get the actual probabilities.

In [13]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions

tensor([[2.2116e-04, 9.9978e-01],
        [9.5785e-01, 4.2146e-02]], grad_fn=<SoftmaxBackward0>)

In [14]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}